# R-CNN Implementation #
### Modified code from https://www.pyimagesearch.com/2020/07/13/r-cnn-object-detection-with-keras-tensorflow-and-deep-learning/ by Adrian Rosebrock ###

In [ ]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import numpy as np
import cv2
import json

# Getting dataset filepaths

In [ ]:
dataset_path = './clips'
folder_path = os.listdir(dataset_path)
imageset_path = [os.path.join(dataset_path, x) for x in folder_path]

clean_dataset_path = './clean_dataset'
positive_path = os.path.join(clean_dataset_path, "CAR")
negative_path = os.path.join(clean_dataset_path, "NO_CAR")

# Creating the training set
### High computation time - Do not run. ###

In [ ]:
MAX_PROPOSALS = 2000
MAX_POSITIVE = 5
MAX_NEGATIVE = 5
positive_count = 0
negative_count = 0

imageDataset = []
labelSet = []
for path in imageset_path:
    annotation = os.path.join(path, 'annotation.json')
    groundTruth = []
    xStart, yStart, xEnd, yEnd = getCar(annotation) # top, bottom, left, right
    groundTruth.extend((xStart, yStart, xEnd, yEnd)) # left, top, right, bottom
    imagePath = os.path.join(path, 'imgs/040.jpg')
    img_40 = cv2.imread(imagePath)

    # Getting proposal boxes
    ss = cv2.ximgproc.segmentation.createSelectiveSearchSegmentation()
    ss.setBaseImage(img_40)
    ss.switchToSelectiveSearchFast()
    rects = ss.process()
    proposedRects = []
    for (x, y, w, h) in rects:
        proposedRects.append((x, y, x + w, y + h)) # left, top, right, bottom
        
    positive = 0
    negative = 0
    
    # Begin selective search
    for proposedRect in proposedRects[:1000]:
        (xStart, yStart, xEnd, yEnd) = proposedRect
        iou = computeIoU(groundTruth, proposedRect)

        roi = None

        # if iou > 0.9 => positive else if iou < 0.05 => negative
        if iou > 0.9 and positive < MAX_POSITIVE:
            roi = img_40[yStart:yEnd, xStart:xEnd]
            roi = cv2.resize(roi, (224, 224))
            filename = "{}.jpg".format(positive_count)
            outputPath = os.path.join(positive_path, filename)
            cv2.imwrite(outputPath, roi)
            positive_count += 1
            positive += 1
        
        if iou < 0.05 and negative < MAX_NEGATIVE:
            roi = img_40[yStart:yEnd, xStart:xEnd]
            roi = cv2.resize(roi, (224, 224))
            filename = "{}.jpg".format(negative_count)
            outputPath = os.path.join(negative_path, filename)
            cv2.imwrite(outputPath, roi)
            negative_count += 1
            negative += 1

# Load dataset and create labels for positive and negative examples #

In [ ]:
data = []
labels = []

count = 0

positiveFolder = os.listdir(positive_path)
negativeFolder = os.listdir(negative_path)
positivePath = [os.path.join(positive_path, x) for x in positiveFolder]
negativePath = [os.path.join(negative_path, x) for x in negativeFolder]

# Resize positive example and append with label as CAR
for path in positivePath:
    img = load_img(path, target_size=(224, 224))
    img = img_to_array(img)
    img = preprocess_input(img)
    data.append(img)
    label = path.split(os.path.sep)[-2]
    labels.append(label)

# Resize negative example and append with label as NO_CAR
for path in negativePath:
    count += 1
    if (count == 400): break # break at 400 negative examples (half of positive examples)
    img = load_img(path, target_size=(224, 224))
    img = img_to_array(img)
    img = preprocess_input(img)
    data.append(img)
    label = path.split(os.path.sep)[1]
    labels.append(label)

# Functions for re-use

# Retrieving bbox dimensions 

In [ ]:
def getCar(annotations):
    # Open annotation.json
    f = open(annotations, 'r')
    train = json.load(f)

    f.close()
    # Getting bbox coordinates
    top = int(train[0]["bbox"]["top"])
    bottom = int(train[0]["bbox"]["bottom"])
    left = int(train[0]["bbox"]["left"])
    right = int(train[0]["bbox"]["right"])

    return left, top, right, bottom

# Computes intersection of union ratio #
## Adapted from Rosebrock's tutorial ##

In [ ]:
def computeIoU(A, B):
	# determine the (x, y)-coordinates of the intersection rectangle
	X1 = max(A[0], B[0]) # left
	Y1 = max(A[1], B[1]) # top
	X2 = min(A[2], B[2]) # right
	Y2 = min(A[3], B[3]) # bottom

	# compute the area of intersection rectangle
	interArea = max(0, X2 - X1 + 1) * max(0, Y2 - Y1 + 1)

	# compute the area of both the prediction and ground-truth
	area_A = (A[2] - A[0] + 1) * (A[3] - A[1] + 1)
	area_B = (B[2] - B[0] + 1) * (B[3] - B[1] + 1)
	# compute the intersection over union by taking the intersection
	# area and dividing it by the sum of prediction + ground-truth
	# areas - the intersection area
	iou = interArea / float(area_A + area_B - interArea)

	# return the intersection over union value
	return iou

# Performs non-max suppression to remove overlapping bboxes #
## Adapted from Rosebrock's tutorial ##

In [ ]:
def non_max_suppression(boxes, overlapThresh):
    if len(boxes) == 0:
        return []
    pick = []
    x1 = boxes[:,0]
    y1 = boxes[:,1]
    x2 = boxes[:,2]
    y2 = boxes[:,3]
    area = (x2 - x1 + 1) * (y2 - y1 + 1)
    idxs = np.argsort(y2)

    while len(idxs) > 0:
        last = len(idxs) - 1
        i = idxs[last]
        pick.append(i)
        suppress = [last]

        for pos in range(0, last):
            j = idxs[pos]
            xx1 = max(x1[i], x1[j])
            yy1 = max(y1[i], y1[j])
            xx2 = min(x2[i], x2[j])
            yy2 = min(y2[i], y2[j])
            w = max(0, xx2 - xx1 + 1)
            h = max(0, yy2 - yy1 + 1)
            overlap = float(w * h) / area[j]
            if overlap > overlapThresh:
                suppress.append(pos)
        idxs = np.delete(idxs, suppress)
    return boxes[pick]

# Flexible parameters for experimentation #

In [ ]:
lr = 0.01 # learning rate
BS = 32 # batch size
EPOCHS = 5 # num of epochs

# CNN Architecture #
## Adapted from Rosebrock's Tutorial ##

In [ ]:
baseModel = MobileNetV2(weights="imagenet", include_top=False,
	input_tensor=Input(shape=(224, 224, 3)))
headModel = baseModel.output
headModel = AveragePooling2D(pool_size=(7, 7))(headModel)
headModel = Flatten(name="flatten")(headModel)
headModel = Dense(128, activation="relu")(headModel)
headModel = Dropout(0.5)(headModel)
headModel = Dense(2, activation="softmax")(headModel)
model = Model(inputs=baseModel.input, outputs=headModel)
for layer in baseModel.layers:
	layer.trainable = False

In [ ]:
opt = Adam(lr=lr)
model.compile(loss="binary_crossentropy", optimizer=opt,
	metrics=["accuracy"])
H = model.fit(X_train, y_train,
	steps_per_epoch=len(X_train) // BS,
	validation_data=(X_test, y_test),
	validation_steps=len(X_test) // BS,
	epochs=EPOCHS)

In [ ]:
imageDataset = np.array(data, dtype = "float32")
labelSet = np.array(labels)

lb = LabelBinarizer()
labelSet = lb.fit_transform(labelSet)
labelSet = to_categorical(labelSet)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(imageDataset, labelSet, test_size = 0.33)

# Evaluation of RCNN

In [ ]:
img = cv2.imread('./test_image.png') # image to test on

# Get proposals
ss = cv2.ximgproc.segmentation.createSelectiveSearchSegmentation()
ss.setBaseImage(img)
ss.switchToSelectiveSearchFast()
rects = ss.process()

In [ ]:
# initialize the list of region proposals that we'll be classifying
# along with their associated bounding boxes
proposals = []
boxes = []
# loop over the region proposal bounding box coordinates generated by
# running selective search
for (x, y, w, h) in rects[:2000]:
	roi = img[y:y + h, x:x + w]
	roi = preprocess_input(roi)
	# update our proposals and bounding boxes lists
	proposals.append(roi)
	boxes.append((x, y, x + w, y + h))

In [ ]:
# convert the proposals and bounding boxes into NumPy arrays
proposals = np.array(proposals, dtype="float32")
boxes = np.array(boxes, dtype="int32")
# classify each of the proposal ROIs using fine-tuned model
proba = model.predict(proposals)

In [ ]:
# create copies for quick re-use/experimenting
cloneBoxes = boxes
cloneProb = proba

In [ ]:
# get result for each detection
labels = np.argmax(cloneProb, axis = 1)
# take results that are cars == 0
found = np.where(labels == 0)[0]
cloneBoxes = cloneBoxes[found]
cloneProb = cloneProb[found][:, 1]
# further filter indexes using a minimum probability
found = np.where(1-cloneProb > 0.9999999)
cloneBoxes = cloneBoxes[found]
cloneProb = cloneProb[found]

In [1]:
# perform non-max suppression to remove overlapping bboxes
finalBox = non_max_suppression(cloneBoxes, 0.1)

NameError: name 'non_max_suppression' is not defined

In [ ]:
img = cv2.imread('./test_image.png')
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
# draw bboxes on detected cars
for box in finalBox:
    (startX, startY, endX, endY) = box
    cv2.rectangle(img, (startX, startY), (endX, endY), (0, 255, 0), 2)

In [ ]:
# visualisation
plt.imshow(img)
plt.show()